# 为任意对象做深复制和浅复制
深复制的副本不会共享内部对象的引用，copy 模块提供的 deepcopy 和 copy 函数能为任意对象做深复制和浅复制

In [11]:
class Bus:
    
    def __init__(self, passengers = None) -> None:
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)


In [12]:
import copy
bus1 = Bus(['Alice', 'Bob', 'Claire'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)
# id 不同，它们是不同的对象

(3041326694944, 3041326697584, 3041326696960)

In [4]:
bus1.drop('Bob')
bus2.passengers
# bus1 与 bus2 的 passengers 引用相同

['Alice', 'Claire']

In [5]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(2294739778176, 2294739778176, 2294739751168)

In [6]:
# 但 bus3 不同，它是 bug1 的深复制
bus3.passengers

['Alice', 'Bob', 'Claire']

In [9]:
# 循环引用：b 引用 a，然后再追加到 a 中； deepcopy 会想办法复制 a
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [10]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

深复制有可能太深了，对象可能会引用不该复制的外部资源或单例值。我们可以实现特殊方法\_\_copy__() 和 \_\_deepcopy__()，控制 copy 和 deepcopy 的行为

# 8.4 函数的参数作为引用时

In [11]:
# 函数可能会修改收到的任何对象
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)


3

In [12]:
(x, y)

(1, 2)

In [13]:
a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)

[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]


## 8.4.1 不要使用可变类型作为参数的默认值

In [20]:
class HauntedBus:
    """备受幽灵乘客折磨的校车"""

    def __init__(self, passengers = []):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [22]:
bus1 = HauntedBus(['Alice', 'Bob'])
bus1.passengers

['Alice', 'Bob']

In [23]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bob', 'Charlie']

In [27]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie', 'Carrie', 'Carrie']

In [28]:
# 这里就出问题了，bus2 与 bus3 的 passengers 为同一个引用
bus3 = HauntedBus()
bus3.passengers

['Carrie', 'Carrie', 'Carrie']

In [29]:
bus3.pick('Dive')
bus2.passengers

['Carrie', 'Carrie', 'Carrie', 'Dive']

In [30]:
bus2.passengers is bus3.passengers

True

In [31]:
bus1.passengers is bus2.passengers

False

实例化 HauntedBus 时，若传入乘客，会按预期运作，但是不为 HauntedBus 指定乘客的话， self.passengers 会变成 passengers 的别名。  
这样的原因是，默认值在定义函数时计算(通常在加载模块时)，因此默认值变成了函数对象的属性。因此，如果默认值是可变对象，且修改了它的值，那么后续的函数调用都会受影响

In [33]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [34]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Carrie', 'Carrie', 'Dive'],)

In [35]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

可变默认值导致的这个问题说明了为什么通常使用 None 作为接收可变值的参数的默认值

## 8.4.2 防御可变参数

In [37]:
class TwilightBus:
    
    def __init__(self, passengers = None) -> None:
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)


In [38]:
basketball_team = ['sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('sue')
basketball_team

['Maya', 'Diana', 'Pat']

在对象进行下车操作后，原本作为参数的列表也改变了，这是因为在定义时让 self.passengers 称为了 passengers 的别名  
要避免这种情况，应使用 self.passengers = list(passengers) 的方法来复制一份

# 8.5 del 和垃圾回收
del 语句删除名称，而不是对象，del命令可能会导致对象当作垃圾被回收，但是仅当删除的变量保存的是对象的最后一个引用，或者无法获得对象时。重新绑定也可能会导致对象的引用数量归零，导致对象被销毁。

有个 \_\_del__ 特殊方法，但是它不会销毁实例，不应该在代码中被调用。即将销毁实例时， Python 解释器会调用 \_\_del__ 方法，给实例最后的机会，释放外部资源。

In [1]:
# 没有指向对象的引用时，监视对象生命结束时的情形
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('bye')

ender = weakref.finalize(s1, bye) # 在 s1 引用的对象上注册 bye 回调
ender.alive

True

In [2]:
del s1
ender.alive

True

In [3]:
s2 = '1'    # 此视再无引用指向 {1, 2, 3} 它被回收

bye


In [4]:
ender.alive

False

# 8.6 弱引用
弱引用不会增加对象的引用数量。引用的目标对象称为 __所指对象__   
下面示例展示了如何使用 weakref.ref 实例获取所指对象。如果对象村炸，调用弱引用可以获取对象，否则返回 None

In [5]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x0000022359A2F040; to 'set' at 0x00000223599EE580>

In [6]:
wref()

{0, 1}

In [7]:
a_set = {2, 3, 4}
wref()

{0, 1}

In [8]:
wref() is None

False

In [10]:
wref() is None

False

在这里需要注意，控制台会自动把 _ 变量绑定到结果不为 None 的表达式结果上，这对演示的结果产生了影响。  
也凸显了一个实际问题，微观管理内存时，往往会得到意外的结果，因为不明显的隐式复制会为对象创建新引用。调用跟踪对象也常导致意料之外的调用

## 8.6.1 WeakValueDictionary
WeakValueDictionary 类实现的是一种可变映射， 里面的值是对象的弱引用。被引用的对象在程序中的其他地方被当作垃圾回收后，对应的键会从 WeakValueDictionary 中删除。因此，WeakValueDictionary 常用于缓存

In [15]:
class Cheese:
    def __init__(self, kind) -> None:
        self.kind = kind
        pass

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [34]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('R'), Cheese('T'), Cheese('B'), Cheese('P')]
for chess in catalog:
    stock[chess.kind] = Cheese

sorted(stock.keys())

['B', 'P', 'R', 'T']

In [35]:
del catalog
sorted(stock.keys())
# 此处应只剩下最后一项，因为前面三个的引用都被释放了，而在 for chess in catalog 中 chess 是全局变量，除非显示消除，否则不会消失

['B', 'P', 'R', 'T']

与 WeakValueDictionary 对应的是 WeakKeyDictionary，后者的键是弱引用，它可以为应用中其他部分拥有的对象附加数据，这样就无需为对象添加属性。这对覆盖属性访问权限的对象尤其有用  
weakref 模块还提供了 WeakSet 类，它可以保存元素弱引用的集合。元素没有强引用时，集合会把它删除。如果一个类需要知道所有实例，一种好的方案是创建一个 WeakSet 类型的类属性，保存实例的引用。

## 8.6.2 弱引用的局限
不是每个 Python 对象都可以作为弱引用的目标。基本上 list 和 dict 实例不能作为所指对象，但他们的子类可以轻松解决这个问题

In [36]:
class MyList(list):
    """list的子类，实例可以作为弱引用的目标"""

a_list= MyList(range(10))

# a_list 可以作为弱引用目标
wref_to_a_list = weakref.ref(a_list)

set 实例可以作为所指对象
但 int 和 tuple 实例不能作为弱引用的目标，甚至它们的子类也不行